# Configuration & Constants

In [0]:
# === Configuration & Constants ===

# Key Vault scope and NetSuite credential keys mapping
VAULT_SCOPE = "azure_key_vault"
NS_CREDENTIAL_KEYS = {
    'account':              'NS-SB-ACCOUNT-ID',
    'realm':                'NS-SB-REALM',
    'consumer_key':         'NS-SB-CONSUMER-KEY',
    'consumer_secret':      'NS-SB-CONSUMER-SECRET',
    'token_key':            'NS-SB-TOKEN-ID',
    'token_secret':         'NS-SB-TOKEN-SECRET'
}
VERSION = "v1"

# Construct ADLS paths for source files and processed log
MEDAL = "bronze"
SUBFOLDER = "paycor"
ENTITY = "journalEntry"
BASE_PATH = f"abfss://{MEDAL}@qydatalake.dfs.core.windows.net/{SUBFOLDER}"
FILE_DIR = f"{BASE_PATH}/{ENTITY}/"
LOG_DIR = f"{BASE_PATH}/logs/{ENTITY}_processed/"

# Set Logging

In [0]:
from datetime import datetime
from zoneinfo import ZoneInfo
import logging
from logging.handlers import BufferingHandler

# Configure the root logger’s level & format
logging.basicConfig(
    level=logging.INFO,
    format= "%(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Grab the root logger once for the whole module
root_logger = logging.getLogger()
root_logger.setLevel(logging.INFO)

2025-07-08 01:47:45 INFO Received command c on object id p1


# Utility Functions

In [0]:
# === Utility Functions ===

import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, DateType, BooleanType, TimestampType
from pyspark.sql.utils import AnalysisException

def list_unprocessed_files(file_path: str, file_log_path: str, extension: str = ".dat"):
    """
    List files in ADLS directory, filter by extension, and exclude those
    already marked as processed in the Delta log table.
    """
    # 1. list files
    files = [(f.name, ) for f in dbutils.fs.ls(file_path) if f.name.endswith(extension)]

    # 2. build DataFrame
    filename_df = spark.createDataFrame(files, ["filename"])

    # 3. read or init log
    try:
        filelog_df = spark.read.format("delta").load(file_log_path) # read processed log if exists
    except AnalysisException:
        log_schema = StructType([
            StructField("filename", StringType(), False),
            StructField("filePath", StringType(), False),
            StructField("isProcessed", BooleanType(), False),
            StructField("processedAt", TimestampType(), False)         
        ])

        filelog_df = spark.createDataFrame([], schema=log_schema) # create an empty processed log if not exists

    # 4. subtract processed
    unprocessed_df = (
        filename_df
        .join(filelog_df.filter(F.col("isProcessed") == True), on="filename", how="left_anti")
    )

    return unprocessed_df

In [0]:
import json

def compose_journal_entry_post_body(file_path: str, schema: StructType) -> dict:
    """
    Read a CSV file for one journal entry, enrich with lookup tables,
    and construct the JSON payload for NetSuite API.
    """
    # Read raw CSV with schema
    raw_df = (
        spark.read
        .format("csv")
        .option("header", True)
        .option("dateFormat", "dd-MMM-yy") # e.g. 28-Mar-25
        .schema(schema)
        .load(file_path)
    )

    # Load and clean lookup tables
    account_df = (
        spark
        .read
        .table("netsuite.bronze.account")
    )

    subsidiary_df = (
        spark
        .read
        .table("netsuite.bronze.subsidiary")
        .withColumn("sub", F.regexp_replace(F.col("name"), r"[^A-Za-z0-9]+", " "))
    )

    department_df = (
        spark
        .read
        .table("netsuite.bronze.department")
    )

    # Normalize and join dimensions
    df = (
        raw_df
        .withColumn("Account", 
            F.when(
                (F.instr("Account", ".") > 0) & (F.length("Account") < 7),
                F.rpad("Account", 7, "0") # right pad Account to len 7 with tailing 0 if it has decimal point, such as 360.02 -> 360.020
            ).otherwise(F.col("Account"))
        )
        .withColumn("sub", F.regexp_replace(
                F.element_at(F.split("Subsidiary", " : "), -1),
                r"[^A-Za-z0-9]+", 
                " "
            )
        )
        .withColumn("depName", F.element_at(F.split("Cost_Centre", " : "), -1))
        .join(account_df.select("acctNumber", F.col("id").alias("account_id")), on=F.col("Account") == account_df["acctNumber"], how="left")
        .join(subsidiary_df.select("sub", F.col("id").alias("subsidiary_id")), on="sub", how="left")
        .join(department_df.select(F.col("name").alias("depName"), F.col("id").alias("department_id")), on="depName", how="left")
    )

    # Collect rows and validate
    rows = df.collect()
    if not rows:
        logging.error(f"No journal entries found in {file_path}")
        raise ValueError(f"No journal entries found in {file_path}")

    # Pick the first row as the header (Assume one file only contains one journal entry)
    hdr = rows[0]

    # Set the top level attributes
    tranDate = hdr["Date"].strftime("%Y-%m-%d")

    reversalDate = None
    if hdr["Reversal_Date"]:
        reversalDate = hdr["Reversal_Date"].strftime("%Y-%m-%d")

    exchangeRate = 1.0

    custbody_je_approver_id = "4597" # default as Eric

    customForm_id = "149" # default as "Quay Journal Entry"

    # Set the line item level attributes
    items = []

    for row in rows:
        item ={
                "account": {
                    "id": row["account_id"]
                },
                "department": {
                    "id": row["department_id"]
                },
                # "cseg1": {
                #     "refName": row["Cost_Centre"]
                # },
                "memo": row["Line_Memo"]
            }
        
        if row["Debit"] and float(row["Debit"]) != 0.0:
            item["debit"] = row["Debit"]
        else:
            item["credit"] = row["Credit"]
        
        items.append(item)

    # Compose the body
    body = {
        "tranDate": tranDate,
        "externalId": hdr["External_ID"],
        "memo": hdr["Memo"],
        "subsidiary": {
            "id": hdr["subsidiary_id"]
        },
        "currency": {
            "refName": hdr["Currency"]
        },
        "exchangeRate": exchangeRate,
        "custbody_je_approver": {
            "id": custbody_je_approver_id
        },
        "customForm": {
            "id": customForm_id
        },
        "line": {
            "items": items
        }
    }

    if reversalDate:
        body["reversalDate"] = reversalDate

    return body

2025-07-08 01:47:45 INFO Received command c on object id p1


In [0]:
import requests
from requests_oauthlib import OAuth1

def fetch_secret(key: str) -> str:
    """
    Retrieve a secret value from Azure Key Vault using Databricks utilities.
    """
    return dbutils.secrets.get(scope=VAULT_SCOPE, key=NS_CREDENTIAL_KEYS[key])

def set_netsuite_connection(
    version: str = VERSION,
    params: dict = None
):
    """
    Build REST endpoint, OAuth1 auth, headers, and default params.
    """
    creds = {k: fetch_secret(k) for k in NS_CREDENTIAL_KEYS.keys()}
    account = creds["account"]

    # Build the base URL for REST calls
    base_url = f"https://{account}.suitetalk.api.netsuite.com/services/rest/record/{version}/"
    
    # Set authentication
    oauth = OAuth1(
        client_key=creds["consumer_key"],
        client_secret=creds["consumer_secret"],
        resource_owner_key=creds["token_key"],
        resource_owner_secret=creds["token_secret"],
        signature_method="HMAC-SHA256",
        realm=creds["realm"]
    )

    # Standard NetSuite REST headers
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }

    # Set parameters. Clone params per call to avoid shared mutable state in threads
    call_params = dict(params) if params else {}
    
    return base_url, oauth, headers, call_params

def post_netsuite(endpoint: str, body: dict, base_url: str, oauth: OAuth1, headers: dict, params: dict) -> list:
    """
    Send a POST request to NetSuite REST API and validate response.
    """
    # Set url
    url = base_url + endpoint

    try:
        response = requests.post(url, auth=oauth, headers=headers, params=params, json=body)
        response.raise_for_status() # Will raise HTTPError for 4xx / 5xx
    except requests.HTTPError as http_err: # You *will* get here on 4xx/5xx
        status = http_err.response.status_code
        error_message = response.json()
        logging.error(f"NetSuite returned {status} for POST {endpoint}: {error_message}")
        raise
    except requests.RequestException as e: # Network errors, timeouts, etc.
        logging.error(f"Network error POSTing to NetSuite {endpoint}: {e}")
        raise
    
    # only get here on 2xx
    logging.info(f"POST {endpoint} succeeded with {response.status_code}")
    return response.status_code

def log_progress(
    log_records: list     =None,
    medal: str            ="bronze",
    subfolder_path: str   ="paycor", 
    entity: str           =""
):
    """
    Persist captured logs into Delta in ADLS
    """
    if not log_records:
        return

    # Define the log path
    log_path = f"abfss://{medal}@qydatalake.dfs.core.windows.net/{subfolder_path}/logs/{entity}_log"

    # Define the log schema
    schema = StructType([
        StructField("run_ts", TimestampType(), nullable=False),
        StructField("level", StringType(), nullable=False),
        StructField("message", StringType(), nullable=False)
    ])

    # Create the log dataframe with exact one file output
    log_df = spark.createDataFrame(log_records, schema).coalesce(1)

    # Write the log
    (
        log_df
        .write
        .format("delta")
        .mode("append")
        .option("mergeSchema", True)
        .save(log_path)
    )

    logging.info("Logged %d entries for %s", len(log_records), entity)

2025-07-08 01:47:45 INFO Received command c on object id p1
2025-07-08 01:47:45 INFO Received command c on object id p0


In [0]:
# === MAIN PIPELINE ===

from delta.tables import DeltaTable

def main():
    """
    Main pipelines: gathers unprocessed journal entry files from ADLS, processes them, and posts.
    """
    try:
        # Identify new files to process
        unprocessed_df = list_unprocessed_files(FILE_DIR, LOG_DIR)

        # Log if no files to process
        if unprocessed_df.rdd.isEmpty():
            logging.info("No new journal entry files found to process.")
            return

        # Define schema for journal entry CSVs
        schema = StructType([
            StructField("Date", DateType(), False),
            StructField("External_ID", StringType(), False),
            StructField("Account", StringType(), True),
            StructField("Debit", DoubleType(), True),
            StructField("Credit", DoubleType(), True),
            StructField("Line_Memo", StringType(), True),
            StructField("Subsidiary", StringType(), False),
            StructField("Cost_Centre", StringType(), True),
            StructField("Memo", StringType(), True),
            StructField("Reversal_Date", DateType(), True),
            StructField("Currency", StringType(), False)
        ])

        # Define Processed Log schema
        log_schema = StructType([
            StructField("filename", StringType(), False),
            StructField("filePath", StringType(), False),
            StructField("isProcessed", BooleanType(), False),
            StructField("processedAt", TimestampType(), False)
        ])

        # Processed Log Delta Table
        if not DeltaTable.isDeltaTable(spark, LOG_DIR):
            empty_log = spark.createDataFrame(data=[], schema=log_schema)
            empty_log.coalesce(1).write.format("delta").mode("overwrite").save(LOG_DIR)
        log_tbl = DeltaTable.forPath(spark, LOG_DIR)

        # Set up NetSuite connection
        base_url, oauth, headers, params = set_netsuite_connection()

        # Loop through each unprocessed file
        for row in unprocessed_df.collect():
            filename = row.filename
            file_path = FILE_DIR + filename
            
            try:
                # Compose JSON body for POST
                body = compose_journal_entry_post_body(file_path=file_path, schema=schema)
                logging.info(f"[{filename}] JSON body composed")
                
                # POST to NetSuite
                status_code = post_netsuite(endpoint="journalEntry", body=body, base_url=base_url, oauth=oauth, headers=headers, params=params)
                logging.info(f"[{filename}] POST succeeded: {status_code}")
                is_processed = True

            except Exception as e:
                logging.error(f"[{filename}] processing FAILED: {e}")
                is_processed = False

            # Upsert processing record in Delta log
            new_log_df = spark.createDataFrame(data=[(filename, file_path, is_processed, datetime.now(tz=ZoneInfo("UTC")))], schema=log_schema)
            (
                log_tbl.alias("tar")
                .merge(new_log_df.alias("src"), "tar.filename = src.filename")
                .whenMatchedUpdateAll()
                .whenNotMatchedInsertAll()
                .execute()
            )
            logging.info(f"[{filename}] marked the process status in log")
    
    except Exception as e:
        # Catch any unexpected exception, log it, but don’t skip the finally block
        logging.exception(f"Unexpected failure in main pipeline: {e}")
    
    finally: # Always write out whatever got captured in buffered logs 
        # Extract buffered logs and persist
        entity_logs = [
            {
                "run_ts": datetime.fromtimestamp(rec.created, tz=ZoneInfo("UTC")),
                "level": rec.levelname,
                "message": rec.getMessage()
            }
            for rec in buffer_handler.buffer
        ]
        # save the logs
        log_progress(log_records=entity_logs, medal="bronze", subfolder_path="paycor", entity="journalEntry")

2025-07-08 01:47:45 INFO Received command c on object id p1


In [0]:
# === RUN PIPELINE ===

if __name__ == "__main__":
    # Set buffered logging: Attach the buffered ListHandler before main() so that event logs in main() can be logged
    buffer_handler = BufferingHandler(capacity=10000)
    buffer_handler.setLevel(logging.INFO)
    root_logger.addHandler(buffer_handler)
    
    # Run
    main()

# --- End of post_netsuite_journal_entry.py ---

2025-07-08 01:47:46 INFO Received command c on object id p1
2025-07-08 01:47:46 INFO Received command c on object id p0
2025-07-08 01:47:47 INFO Received command c on object id p0
2025-07-08 01:47:48 INFO Received command c on object id p0
2025-07-08 01:47:49 INFO Python Server ready to receive messages
2025-07-08 01:47:49 INFO Received command c on object id p0
2025-07-08 01:47:49 INFO Received command c on object id p0
2025-07-08 01:47:50 INFO Received command c on object id p0
2025-07-08 01:47:51 INFO [GL_000175185_1_20250326201544953.dat] JSON body composed
2025-07-08 01:47:51 INFO Received command c on object id p0
2025-07-08 01:47:51 INFO Received command c on object id p0
2025-07-08 01:47:51 INFO Received command c on object id p0
2025-07-08 01:47:51 INFO Received command c on object id p0
2025-07-08 01:47:51 INFO Received command c on object id p0
2025-07-08 01:47:52 INFO Received command c on object id p0
2025-07-08 01:47:53 INFO Received command c on object id p0
2025-07-08 0